In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
csv_file_path = r'C:\Users\gijsv\OneDrive\Documenten\Ironhack\week3\project\Data\GDP\CountriesGDP1960-2020.csv'
df_original = pd.read_csv(csv_file_path)
df_gdp = df_original.copy()

df_gdp.rename(columns={'Country Name': 'country', 'Year' : 'year', 'Value' : 'gdp'}, inplace=True)
columns_to_drop = [str(year) for year in range(1960, 1980)] +  ['Country Code']
df_gdp = df_gdp.drop(columns=columns_to_drop, errors='ignore')
melted_df = df_gdp.melt(id_vars=['country'], var_name='year', value_name='gdp')
melted_df['year'] = melted_df['year'].astype(int)
melted_df['year'] = melted_df['year'] // 5 * 5
grouped_df = melted_df.groupby(['country', 'year']).agg({'gdp': 'first'}).reset_index()
grouped_df['gdp_%_change'] = grouped_df.groupby('country')['gdp'].pct_change() * 100
grouped_df.head(25)

,country,year,gdp,gdp_%_change
0,Africa Eastern and Southern,1980,1.570000e+11,NaN
1,Africa Eastern and Southern,1985,1.300000e+11,-17.197452
2,Africa Eastern and Southern,1990,2.120000e+11,63.076923
3,Africa Eastern and Southern,1995,2.700000e+11,27.358491
4,Africa Eastern and Southern,2000,2.840000e+11,5.185185
5,Africa Eastern and Southern,2005,5.120000e+11,80.281690
6,Africa Eastern and Southern,2010,8.470000e+11,65.429688
7,Africa Eastern and Southern,2015,9.200000e+11,8.618654
8,Africa Eastern and Southern,2020,9.210000e+11,0.108696
9,Africa Western and Central,1980,1.120000e+11,NaN


In [3]:
csv_file_path = r'C:\Users\gijsv\OneDrive\Documenten\Ironhack\week3\project\Data\Education\OUP_long_MF2564_v1.csv'

df_original = pd.read_csv(csv_file_path)
df_edu = df_original.copy()
df_edu = df_edu[df_edu['year'] >= 1980]
df_edu['country'] = df_edu['country'].replace({'USA': 'United States'})

In [11]:
merged_df = pd.merge(grouped_df, df_edu, on=['country', 'year'], how='inner')
merged_df = merged_df.drop(columns=['BLcode', 'WBcode' , 'region_code', 'sex'] , errors='ignore')

#Add percentual changes to education data
merged_df['gdp_%_change'] = merged_df.groupby('country')['gdp'].pct_change() * 100
merged_df['gdp_per_capita'] = merged_df['gdp'] / (merged_df['pop']*1000)
merged_df['gdp_%_change_capita'] = merged_df.groupby('country')['gdp_per_capita'].pct_change() * 100

merged_df['yr_sch_%'] = merged_df.groupby('country')['yr_sch'].pct_change() * 100
merged_df['yr_sch_pri_%'] = merged_df.groupby('country')['yr_sch_pri'].pct_change() * 100
merged_df['yr_sch_sec_%'] = merged_df.groupby('country')['yr_sch_sec'].pct_change() * 100
merged_df['yr_sch_ter_%'] = merged_df.groupby('country')['yr_sch_ter'].pct_change() * 100
merged_df['lp_%'] = merged_df.groupby('country')['lp'].pct_change() * 100
merged_df['lpc_%'] = merged_df.groupby('country')['lpc'].pct_change() * 100
merged_df['lsc_%'] = merged_df.groupby('country')['lsc'].pct_change() * 100
merged_df['lhc_%'] = merged_df.groupby('country')['lhc'].pct_change() * 100
merged_df['pop_%'] = merged_df.groupby('country')['pop'].pct_change() * 100

pd.set_option('display.max_columns', None)
# reindex columns
desired_columns = ['year', 'country', 'gdp','gdp_%_change','gdp_per_capita', 'gdp_%_change_capita', 'agefrom', 'ageto', 'lu', 'lp','lp_%', 'lpc','lpc_%', 'ls', 'lsc', 'lsc_%', 'lh', 'lhc','lhc_%', 'yr_sch', 'yr_sch_%','yr_sch_pri', 'yr_sch_pri_%','yr_sch_sec', 'yr_sch_sec_%' ,'yr_sch_ter', 'yr_sch_ter_%', 'pop', 'pop_%']
merged_df = merged_df.reindex(columns=desired_columns)

merged_df.set_index('country', inplace=True)


merged_df.head(10)

,year,gdp,gdp_%_change,gdp_per_capita,gdp_%_change_capita,agefrom,ageto,lu,lp,lp_%,lpc,lpc_%,ls,lsc,lsc_%,lh,lhc,lhc_%,yr_sch,yr_sch_%,yr_sch_pri,yr_sch_pri_%,yr_sch_sec,yr_sch_sec_%,yr_sch_ter,yr_sch_ter_%,pop,pop_%
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Algeria,1980,4.234638e+10,NaN,7309.922463,NaN,25,64,67.7,23.6,NaN,3.7,NaN,7.7,5.4,NaN,1.0,0.6,NaN,1.91,NaN,1.34,NaN,0.54,NaN,0.03,NaN,5793,NaN
Algeria,1985,5.793787e+10,36.818938,8419.978008,15.185599,25,64,58.0,29.5,25.000000,10.5,183.783784,10.4,7.6,40.740741,2.1,1.3,116.666667,2.81,47.120419,1.95,45.522388,0.79,46.296296,0.07,133.333333,6881,18.781288
Algeria,1990,6.204856e+10,7.095004,7445.231935,-11.576587,25,64,47.9,34.4,16.610169,13.9,32.380952,14.2,10.7,40.789474,3.5,2.2,69.230769,3.76,33.807829,2.51,28.717949,1.13,43.037975,0.12,71.428571,8334,21.116117
Algeria,1995,4.176432e+10,-32.690922,4156.480427,-44.172586,25,64,39.5,37.9,10.174419,16.5,18.705036,17.5,13.4,25.233645,5.1,3.3,50.000000,4.61,22.606383,2.99,19.123506,1.46,29.203540,0.17,41.666667,10048,20.566355
Algeria,2000,5.479039e+10,31.189491,4601.527903,10.707316,25,64,33.1,40.4,6.596306,20.5,24.242424,20.1,15.6,16.417910,6.5,4.2,27.272727,5.35,16.052061,3.42,14.381271,1.72,17.808219,0.21,23.529412,11907,18.501194
Algeria,2005,1.030000e+11,87.989162,7255.564948,57.677300,25,64,27.9,44.6,10.396040,25.4,23.902439,18.9,14.8,-5.128205,8.7,5.6,33.333333,5.84,9.158879,3.75,9.649123,1.81,5.232558,0.29,38.095238,14196,19.223986
Algeria,2010,1.610000e+11,56.310680,9497.404436,30.898207,25,64,21.9,46.8,4.932735,28.6,12.598425,20.3,16.1,8.783784,10.9,7.1,26.785714,6.47,10.787671,4.14,10.400000,1.97,8.839779,0.36,24.137931,16952,19.413919
Australia,1980,1.500000e+11,NaN,21496.130696,NaN,25,64,0.9,6.9,NaN,3.5,NaN,69.7,51.5,NaN,22.5,14.6,NaN,11.00,NaN,5.84,NaN,4.42,NaN,0.74,NaN,6978,NaN
Australia,1985,1.810000e+11,20.666667,23479.050461,9.224543,25,64,1.0,7.1,2.898551,3.4,-2.857143,66.1,52.1,1.165049,25.7,16.6,13.698630,11.30,2.727273,5.82,-0.342466,4.63,4.751131,0.85,14.864865,7709,10.475781


In [5]:
correlation = merged_df['gdp_%_change'].corr(merged_df['lhc_%'])
#correlation = merged_df['gdp_%_change'].corr(merged_df['pop_%'])
print(correlation)


-0.060398630023381875


In [7]:
missing_values = merged_df.isnull().sum()
df_clean = merged_df.dropna()

X = df_clean['lhc_%']
y = df_clean['gdp_%_change']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           gdp_%_change   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.201
Date:                Tue, 05 Mar 2024   Prob (F-statistic):              0.274
Time:                        21:23:56   Log-Likelihood:                -1782.9
No. Observations:                 330   AIC:                             3570.
Df Residuals:                     328   BIC:                             3577.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         44.7231      3.453     12.952      0.0